In [42]:
import pandas as pd

url = 'https://campus.il3.ub.edu/pluginfile.php/444355/mod_assign/intro/datos_obesidad.xlsx'


try:
    # Leer el archivo Excel desde la URL
    df = pd.read_excel(url, engine='openpyxl')
    
    # Ahora puedes trabajar con el DataFrame df
    print(df.head())  # Imprime las primeras filas del DataFrame
    
except Exception as e:
    print(f"Error al leer el archivo Excel desde la URL: {e}")

Error al leer el archivo Excel desde la URL: File is not a zip file
